<a href="https://colab.research.google.com/github/RemyaVKarthikeyan/AA-Stagecoach-Project/blob/main/File_share_28_QSI_points%2C_SWT_dataframe%2C_Summary_metrics_%2C_AWT_dataframe_(revised).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

05/07/2024

In [1]:
import pandas as pd
import requests
from difflib import SequenceMatcher
from datetime import datetime, timedelta
import pytz
import time

# Initialize empty DataFrame for AWT data
awt_data = pd.DataFrame(columns=['Route_Dir_QSI_No','Stop Point ID', 'AWT','No_Buses_Obs','EWT'])

# Function to normalize stop names
def normalize_stop_name(name):
    return ' '.join(name.lower().split())

# Function to fetch data from the API
def fetch_data(url):
    response = requests.get(url)
    return response.json()

# Function to extract schedule names
def extract_schedule_names(data, schedule_names_dict={}):
    if isinstance(data, dict):
        if data.get('$type') == "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities" and 'knownJourneys' in data:
            if 'name' in data:
                schedule_names_dict[data['name']] = data['knownJourneys']
        for key, value in data.items():
            extract_schedule_names(value, schedule_names_dict)
    elif isinstance(data, list):
        for item in data:
            extract_schedule_names(item, schedule_names_dict)
    return schedule_names_dict

# Function to categorize journeys into hourly slots
def categorize_into_slots(timetable):
    slots = [[] for _ in range(24)]
    for journey in timetable:
        hour = int(journey['hour'])  # Convert hour to integer
        if 0 <= hour < 24:  # Ensure hour is within the valid range
            slots[hour].append(journey)
    return slots

# Function to fetch the current day of the week
def get_day_of_week():
    bst = pytz.timezone('Europe/London')
    now = datetime.now(bst)
    return now.strftime('%A')  # %A gives full weekday name (e.g., 'Monday')

# Function to retrieve stop names from TfL API and match with Route_Dir_QSI_No
def find_route_details(lineID, df):
    # Ensure the 'Route_Dir_QSI_No' column exists
    if 'Route_Dir_QSI_No' not in df.columns:
        print("The 'Route_Dir_QSI_No' column is not present in the provided file.")
        return

    # Convert the lineID to uppercase to ensure case-insensitivity
    lineID = lineID.upper()

    # Convert the 'Route_Dir_QSI_No' column to uppercase for comparison
    df['Route_Dir_QSI_No'] = df['Route_Dir_QSI_No'].str.upper()

    # Normalize the stop names in the DataFrame
    df['STOP_NAME'] = df['STOP_NAME'].apply(normalize_stop_name)

    # Filter the DataFrame based on the lineID
    pattern_A = f"^{lineID}_A\\d+$"  # Regular expression for D7_A**
    pattern_B = f"^{lineID}_B\\d+$"  # Regular expression for D7_B**

    # Filter rows where the 'Route_Dir_QSI_No' column matches the pattern
    filtered_df_A = df[df['Route_Dir_QSI_No'].str.match(pattern_A, na=False)]
    filtered_df_B = df[df['Route_Dir_QSI_No'].str.match(pattern_B, na=False)]

    # Function to fetch and process route sequence data from TfL API
    def fetch_and_process_route_data(route_type, pattern, filtered_df):
        api_url = f"https://api.tfl.gov.uk/Line/{lineID}/Route/Sequence/{route_type}"
        response = requests.get(api_url)

        results_list = []

        if response.status_code == 200:
            route_data = response.json()

            # Iterate through each stop in the route data
            for stop in route_data['stopPointSequences'][0]['stopPoint']:
                stop_name_api = normalize_stop_name(stop['name'])
                stop_id = stop['id']

                # Check if the stop_name_api exists in the filtered DataFrame for the correct direction
                matched_row = filtered_df[(filtered_df['STOP_NAME'] == stop_name_api) &
                                          (filtered_df['Route_Dir_QSI_No'].str.match(pattern))]

                if not matched_row.empty:
                    route_dir_qsi_no = matched_row.iloc[0]['Route_Dir_QSI_No']
                    results_list.append({
                        'Route_Dir_QSI_No': route_dir_qsi_no,
                        'STOP_Name': stop['name'],
                        'ID': stop_id
                    })
                else:
                    # If exact match not found, try partial matching based on words before and after '/'
                    api_stop_name_parts = stop_name_api.split('/')
                    for index, row in filtered_df.iterrows():
                        df_stop_name_parts = row['STOP_NAME'].split('/')
                        for api_part in api_stop_name_parts:
                            for df_part in df_stop_name_parts:
                                if SequenceMatcher(None, df_part.strip(), api_part.strip()).ratio() > 0.8:
                                    matched_row = pd.DataFrame([row])
                                    break
                            if not matched_row.empty:
                                break
                        if not matched_row.empty:
                            break

                    if not matched_row.empty:
                        route_dir_qsi_no = matched_row.iloc[0]['Route_Dir_QSI_No']
                        results_list.append({
                            'Route_Dir_QSI_No': route_dir_qsi_no,
                            'STOP_Name': stop['name'],
                            'ID': stop_id
                        })
        else:
            print(f"Failed to fetch route sequence data from TfL API for {route_type} route. Status code: {response.status_code}")

        return results_list

    # Fetch and process outbound route data for _A**
    matched_results_A = fetch_and_process_route_data('outbound', pattern_A, filtered_df_A)

    # Fetch and process inbound route data for _B**
    matched_results_B = fetch_and_process_route_data('inbound', pattern_B, filtered_df_B)

    # Create DataFrames from the matched results for each direction
    matched_results_df_A = pd.DataFrame(matched_results_A)
    matched_results_df_B = pd.DataFrame(matched_results_B)

    # Function to remove partial matches if exact matches are found
    def remove_partial_matches(exact_df, matched_df):
        for index, row in exact_df.iterrows():
            exact_stop_name = row['STOP_NAME']
            route_dir_qsi_no = row['Route_Dir_QSI_No']
            # Find exact matches in matched_df
            exact_matches = matched_df[(matched_df['Route_Dir_QSI_No'] == route_dir_qsi_no) &
                                       (matched_df['STOP_Name'].apply(normalize_stop_name) == exact_stop_name)]
            if not exact_matches.empty:
                # Remove partial matches
                matched_df = matched_df[~((matched_df['Route_Dir_QSI_No'] == route_dir_qsi_no) &
                                          (matched_df['STOP_Name'].apply(normalize_stop_name) != exact_stop_name))]
        return matched_df

    # Remove partial matches for direction A
    matched_results_df_A = remove_partial_matches(filtered_df_A, matched_results_df_A)

    # Remove partial matches for direction B
    matched_results_df_B = remove_partial_matches(filtered_df_B, matched_results_df_B)

    # Remove duplicate stop names with the same Route_Dir_QSI_No and different IDs
    matched_results_df_A = matched_results_df_A.drop_duplicates(subset=['Route_Dir_QSI_No', 'STOP_Name'], keep='first')
    matched_results_df_B = matched_results_df_B.drop_duplicates(subset=['Route_Dir_QSI_No', 'STOP_Name'], keep='first')

    # Print the matched results for direction A
    print(f"\n\n\033[1m\033[4mQSI stop points for direction {lineID}_A\033[0m\n")
    matched_results_df_A = matched_results_df_A[matched_results_df_A['Route_Dir_QSI_No'].str.match(pattern_A)]
    print(matched_results_df_A[['Route_Dir_QSI_No', 'STOP_Name', 'ID']])

    # Print the matched results for direction B
    print(f"\n\n\033[1m\033[4mQSI stop points for direction {lineID}_B\033[0m\n")
    matched_results_df_B = matched_results_df_B[matched_results_df_B['Route_Dir_QSI_No'].str.match(pattern_B)]
    print(matched_results_df_B[['Route_Dir_QSI_No', 'STOP_Name', 'ID']])

    # Concatenate the matched results DataFrames for directions A and B
    combined_df = pd.concat([matched_results_df_A, matched_results_df_B], ignore_index=True)

    # DataFrames to store SWT data
    swt_data = {
        'Route_Dir_QSI_No': [],
        'ID': [],
        'SWT': [],
        'No_Buses_Sch': []
    }

    # Fetch timetable for each stop point ID and calculate SWT
    bst = pytz.timezone('Europe/London')
    current_hour = datetime.now(bst).hour
    day_of_week = get_day_of_week()

    # Store selected schedule name to ensure it's printed only once
    selected_schedule_name = None
    printed_schedule_name = False

    # Track printed timetable stop IDs
    printed_timetable_stop_ids = []

    while True:
        # Clear previous awt_data DataFrame
        awt_data = pd.DataFrame(columns=['Route_Dir_QSI_No','Stop Point ID','AWT','No_Buses_Obs','EWT'])

        # Update current time and hour
        now = datetime.now(bst)
        current_hour = now.hour
        day_of_week = get_day_of_week()

        # Clear previous SWT data
        swt_data = {
            'Route_Dir_QSI_No': [],
            'ID': [],
            'SWT': [],
            'No_Buses_Sch': []
        }

        for index, row in combined_df.iterrows():
            stop_point_id = row['ID']
            route_dir_qsi_no = row['Route_Dir_QSI_No']

            if f"{lineID}_A" in route_dir_qsi_no:
                direction = 'outbound'
            elif f"{lineID}_B" in route_dir_qsi_no:
                direction = 'inbound'
            else:
                print(f"Invalid route direction for Route_Dir_QSI_No: {route_dir_qsi_no}")
                continue

            url = f'https://api.tfl.gov.uk/Line/{lineID}/Timetable/{stop_point_id}?direction={direction}'
            data = fetch_data(url)

            schedule_names_dict = extract_schedule_names(data)

            if not selected_schedule_name:
                if day_of_week.lower() in ['monday', 'tuesday', 'wednesday', 'thursday']:
                    preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Thursday', 'Monday to Friday']
                elif day_of_week.lower() == 'friday':
                    preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Friday', 'Friday']
                elif day_of_week.lower() == 'saturday':
                    preferred_schedule_names = ['Saturday']
                elif day_of_week.lower() == 'sunday':
                    preferred_schedule_names = ['Sunday']
                else:
                    preferred_schedule_names = [day_of_week]

                for preferred_name in preferred_schedule_names:
                    if preferred_name in schedule_names_dict:
                        selected_schedule_name = preferred_name
                        break

            if selected_schedule_name and not printed_schedule_name:
                print(f"\nToday is {day_of_week}. The selected Schedule name is {selected_schedule_name}.")
                printed_schedule_name = True

            if selected_schedule_name:
                timetable = schedule_names_dict[selected_schedule_name]
                slots = categorize_into_slots(timetable)

                # Calculate Scheduled Wait Time (SWT)
                total_buses_this_hour = len(slots[current_hour])
                if total_buses_this_hour > 0:
                    scheduled_wait_time = 60 / (total_buses_this_hour * 2)  # SWT formula
                else:
                    scheduled_wait_time = float('inf')  # Handle division by zero scenario (though unlikely)

                # Store SWT data
                swt_data['Route_Dir_QSI_No'].append(route_dir_qsi_no)
                swt_data['ID'].append(stop_point_id)
                swt_data['SWT'].append(scheduled_wait_time)
                swt_data['No_Buses_Sch'].append(total_buses_this_hour)

                # Fetch arrival predictions based on SWT data for printed timetable stop IDs
                if stop_point_id in printed_timetable_stop_ids:
                    # Fetch arrival predictions
                    arrival_predictions_df, station_name = fetch_arrival_predictions(lineID, stop_point_id, direction)

                    if arrival_predictions_df is not None and not arrival_predictions_df.empty:
                        #print(f"\nArrival Predictions for stop point {stop_point_id} ({station_name}):")
                        #print(arrival_predictions_df.to_string(index=False))
                        print(f"   ")
                        # Calculating summary metrics
                        total_wawt = arrival_predictions_df['WAWT'].sum()
                        min_arrival = arrival_predictions_df['Expected Arrival (BST)'].min().replace(second=0, microsecond=0)
                        max_arrival = arrival_predictions_df['Expected Arrival (BST)'].max().replace(second=0, microsecond=0)
                        time_diff_minutes = (max_arrival - min_arrival).total_seconds() / 60
                        num_buses_observed = arrival_predictions_df['Vehicle ID'].nunique()

                        # Calculating AWT, SWT, and EWT
                        nbph = swt_data['No_Buses_Sch'][swt_data['ID'].index(stop_point_id)]
                        swt = swt_data['SWT'][swt_data['ID'].index(stop_point_id)]
                        awt = round(total_wawt / time_diff_minutes, 2) if time_diff_minutes > 0 else 0
                        ewt = round(awt - swt, 2)

                        summary_df = pd.DataFrame({
                            'Metric': ['Number of buses scheduled per hour (nbph)', 'Number of buses observed', 'Total WAWT (minutes)',
                                       'Time difference between 1st and last observed buses (minutes)', 'AWT (minutes)', 'SWT (minutes)', 'EWT (minutes)'],
                            'Value': [nbph, num_buses_observed, total_wawt, time_diff_minutes, awt, swt, ewt]
                        })

                        print(f"\nSummary Metrics for {stop_point_id}:")
                        print(summary_df)

                        # Append to awt_data DataFrame
                        awt_data.loc[len(awt_data)] = [swt_data['Route_Dir_QSI_No'][swt_data['ID'].index(stop_point_id)],stop_point_id, awt,num_buses_observed,ewt]

                    # Remove the stop ID from printed_timetable_stop_ids to avoid redundant fetches
                    printed_timetable_stop_ids.remove(stop_point_id)

        # Update printed timetable stop IDs
        printed_timetable_stop_ids = swt_data['ID']

        # Create DataFrame for SWT data
        swt_df = pd.DataFrame(swt_data)
        awt_df = pd.DataFrame(awt_data)
        # Print the SWT DataFrame
        print(f"\n\nSWT DataFrame at hour {current_hour}")
        print(swt_df)

        print(f"\n\nAWT DataFrame at hour {now.strftime('%H:%M')}")
        print(awt_df)

        # Wait for the next 30 seconds
        #clear_output()
        print("\n\nWaiting to fetch updated data...\n\n")
        time.sleep(30)

# Function to fetch arrival predictions with error handling
def fetch_arrival_predictions(line_id, stop_point_id, direction):
    try:
        base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals/{stop_point_id}"
        params = {'direction': direction}
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        if len(data) == 0:
            return pd.DataFrame(), None  # No data available
        station_name = data[0]['stationName']
        predictions = []
        for item in data:
            arrival_time = datetime.strptime(item['expectedArrival'], '%Y-%m-%dT%H:%M:%SZ')
            arrival_time_bst = arrival_time + timedelta(hours=1)
            predictions.append({
                'Line': item['lineName'],
                'Vehicle ID': item['vehicleId'],
                'Stop Point': stop_point_id,
                'Direction': direction,
                'Expected Arrival (BST)': arrival_time_bst,
                'Expected Arrival (HM)': arrival_time_bst.strftime('%H:%M')
            })
        df = pd.DataFrame(predictions)
        df = df.sort_values(by='Expected Arrival (BST)', ascending=True)
        df['Expected Arrival (BST)'] = pd.to_datetime(df['Expected Arrival (BST)'])  # Convert to datetime
        df['Expected Arrival (HM)'] = pd.to_datetime(df['Expected Arrival (HM)'], format='%H:%M')
        df['Headway (minutes)'] = df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60
        df['AWT/bus (minutes)'] = (df['Headway (minutes)'] / 2).round(2)
        df['WAWT'] = (df['Headway (minutes)'] * df['AWT/bus (minutes)']).round(2)
        return df, station_name
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None, None
        time.sleep(30)  # Wait for 30 seconds before fetching data again

# Example usage
if __name__ == "__main__":
    # Load the Excel file into a DataFrame
    file_path = '/content/QSI points.xlsx'  # Modify this path accordingly
    df = pd.read_excel(file_path)

    # Ask the user to enter a lineID
    line_id_input = input("Please enter the lineID: ")

    while True:
        try:
            # Find and display the route details
            find_route_details(line_id_input, df)

            time.sleep(30)

        except KeyboardInterrupt:
            print("\n\nExecution interrupted. Exiting the loop.\n")
            break

Streaming output truncated to the last 5000 lines.
                                              Metric   Value
0          Number of buses scheduled per hour (nbph)    4.00
1                           Number of buses observed    3.00
2                               Total WAWT (minutes)  260.00
3  Time difference between 1st and last observed ...   28.00
4                                      AWT (minutes)    9.29
5                                      SWT (minutes)    7.50
6                                      EWT (minutes)    1.79
   

Summary Metrics for 490010612E:
                                              Metric  Value
0          Number of buses scheduled per hour (nbph)    5.0
1                           Number of buses observed    1.0
2                               Total WAWT (minutes)    0.0
3  Time difference between 1st and last observed ...    0.0
4                                      AWT (minutes)    0.0
5                                      SWT (minutes)    6.0
6   